In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline


In [ ]:
df=pd.read_csv('../input/aurora-roundhacks/new_train.csv')
df1=pd.read_csv('../input/aurora-roundhacks/new_test.csv')

In [ ]:
df['date'].dtypes
df['date'] = pd.to_datetime(df['date'])

df['Year'] = df['date'].dt.year
df['Month'] = df['date'].dt.month
df['Day'] = df['date'].dt.day

df.info()

In [ ]:
df.drop(['date'], axis=1, inplace = True)

In [ ]:
X = df.drop(['temp'], axis=1)

y = df['temp']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [ ]:
df1['date'] = pd.to_datetime(df1['date'])

df1['Year'] = df1['date'].dt.year
df1['Month'] = df1['date'].dt.month
df1['Day'] = df1['date'].dt.day

df1.drop(['date'], axis=1, inplace = True)

df1.info()

In [ ]:
!pip install catboost
!pip install optuna

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgbm

import optuna
from optuna import Trial, visualization

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

In [ ]:
from xgboost import XGBRegressor
from sklearn.model_selection import KFold, cross_val_score

# making 5 splits
kf = KFold(n_splits=5, shuffle=True, random_state=0)

In [ ]:
XGB_params={'colsample_bytree': 1.0,
 'eta': 0.033634380372389666,
 'max_depth': 11,
 'min_child_weight': 6,
 'n_estimators': 500,
 'subsample': 0.8}

In [ ]:
catboost_params={'learning_rate': 0.34572681009606276,
 'max_depth': 13,
 'n_estimators': 150,
 'subsample': 0.9}

In [ ]:
rat_params={'max_depth': 15, 'max_features': 3, 'max_samples': 0.8, 'n_estimators': 1500}

In [ ]:
from sklearn.base import TransformerMixin, BaseEstimator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
import lightgbm as lgbm

In [ ]:
lgbreg = lgbm.LGBMRegressor(device_type='gpu')

# taking positive because cross val score returns -ve values
scores = np.abs(cross_val_score(lgbreg, X, y, cv=kf, scoring='neg_root_mean_squared_error'))

# printing the output
print(f"Scores - {scores}\nMean - {sum(scores)/len(scores)}")

In [ ]:
def Objective(trial):
    params = {
        'lambda_l1': trial.suggest_loguniform('lambda_l1', 1e-8, 12.0),
        'lambda_l2': trial.suggest_loguniform('lambda_l2', 1e-8, 12.0),
        'num_leaves': trial.suggest_int('num_leaves', 2, 256),
        'feature_fraction': trial.suggest_uniform('feature_fraction', 0.4, 1.0),
        'bagging_fraction': trial.suggest_uniform('bagging_fraction', 0.4, 1.0),
        'bagging_freq': trial.suggest_int('bagging_freq', 1, 7),
        'min_child_samples': trial.suggest_int('min_child_samples', 5, 100),
    }
    model = lgbm.LGBMRegressor(**params, device='GPU')
    
    
    # taking positive because cross val score returns -ve values
    scores = np.abs(cross_val_score(model, X, y, cv=kf, scoring='neg_root_mean_squared_error'))
    
    return sum(scores)/len(scores)

In [ ]:
study = optuna.create_study(direction="minimize", study_name='LGBM optimization')
study.optimize(Objective, n_trials=30)

In [ ]:
lgb_params = study.best_params
study.best_params
lgb_params

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.ensemble import StackingRegressor

final_estimator = GradientBoostingRegressor(n_estimators=350, 
                                            random_state=42)

estimators = [('xgb', XGBRegressor(tree_method='gpu_hist', **XGB_params)),
              ('lgb', lgbm.LGBMRegressor(device_type='gpu',**lgb_params)), 
               ('rat',RandomForestRegressor(**rat_params)),
              ('cat', CatBoostRegressor(verbose=0, task_type='GPU', **catboost_params))] 

reg = StackingRegressor(
        estimators=estimators,
         final_estimator=final_estimator)

In [ ]:
# taking positive because cross val score returns -ve values
scores = np.abs(cross_val_score(reg, X, y.values.ravel(), cv=kf, scoring='neg_root_mean_squared_error'))

# printing the output
print(f"Scores - {scores}\nMean - {sum(scores)/len(scores)}")

In [ ]:
reg.fit(X, y.values.ravel())
y_pred = reg.predict(df1)
y_pred=pd.DataFrame(y_pred)

In [ ]:
y_pred.to_csv('Sub12.csv',header=['prediction'],index=False)

In [ ]:
lgb=lgbm.LGBMRegressor(device_type='gpu',lambda_l1= 0.003005391918043113,lambda_l2=8.006108609364085e-07,
num_leaves= 166,feature_fraction= 0.8470174798951228,bagging_fraction= 0.9267780669292209,bagging_freq= 7,min_child_samples= 17)

In [ ]:
from sklearn.model_selection import KFold

accuracy=[]
prediction=0

skf=KFold(n_splits=10, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       lgb.fit(X1_train,Y1_train)
       prediction+=lgb.predict(df1)/10

prediction=pd.DataFrame(prediction)   

In [ ]:
prediction


In [ ]:
prediction.to_csv('Sub13.csv',header=['prediction'],index=False)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rag = RandomForestRegressor()

# taking positive because cross val score returns -ve values
scores = np.abs(cross_val_score(rag, X, y.values.ravel(), cv=kf, scoring='neg_root_mean_squared_error'))

# printing the output
print(f"Scores - {scores}\nMean - {sum(scores)/len(scores)}")

In [ ]:
def Objective(trial):
    params = {
        "n_estimators": trial.suggest_categorical("n_estimators", [50,100,125,250,500,750,1000,1250,1500,2000]),
        "max_depth": trial.suggest_categorical("max_depth",[5,7,9,11,13,15,17]),
        "max_samples": trial.suggest_discrete_uniform("max_samples", 0.1,0.8,0.1),
        "max_features": trial.suggest_categorical("max_features",[1,2,3]),
        "criterion":'mse',
        #"colsample_bylevel": trial.suggest_categorial("colsample_bylevel", [0.3,0.4,0.5,0.6,0.7,0.8])
        # the colsample_bylevel also improves the score a lot, but its commented out because its not supported on GPU currently.
        # If you are interested, do fork the kernel, and try it out. I reached a score of 0.7004 while tuning with it.
        "random_state": 2021
    }
    
    
    model = RandomForestRegressor(**params)
    
    
    # taking positive because cross val score returns -ve values
    scores = np.abs(cross_val_score(model,X , y.values.ravel(), cv=kf, scoring='neg_root_mean_squared_error'))

    return sum(scores)/len(scores)

In [ ]:
study = optuna.create_study(direction="minimize", study_name='RandomForest optimization')
study.optimize(Objective, n_trials=100)

In [ ]:
rat1=study.best_params
study.best_params

In [ ]:
rat_params1={'n_estimators': 1000, 'max_depth': 17, 'max_samples': 0.8, 'max_features': 3}


In [ ]:
rag = RandomForestRegressor(n_estimators= 1000, max_depth= 17, max_samples= 0.8, max_features=3)
rag.fit(X,y)
y_pred=rag.predict(df1)
y_pred=pd.DataFrame(y_pred)

In [ ]:
y_pred.to_csv('Sub14.csv',header=['prediction'],index=False)

In [ ]:
from sklearn.model_selection import KFold

accuracy=[]
prediction1=0

skf=KFold(n_splits=5, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       rag.fit(X1_train,Y1_train)
       prediction1+=rag.predict(df1)/5

prediction1=pd.DataFrame(prediction1)  
prediction1

In [ ]:
prediction1.to_csv('Sub15.csv',header=['prediction'],index=False)

In [ ]:
lgb1={'lambda_l1': 0.003005391918043113,
 'lambda_l2': 8.006108609364085e-07,
 'num_leaves': 166,
 'feature_fraction': 0.8470174798951228,
 'bagging_fraction': 0.9267780669292209,
 'bagging_freq': 7,
 'min_child_samples': 17}

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor
from sklearn.ensemble import StackingRegressor

final_estimator = GradientBoostingRegressor(n_estimators=150, 
                                            random_state=42)

estimators = [('xgb', XGBRegressor(tree_method='gpu_hist', **XGB_params)),
              ('lgb', lgbm.LGBMRegressor(device_type='gpu',**lgb1)), 
               ('rat',RandomForestRegressor(**rat1))] 

reg2 = StackingRegressor(
        estimators=estimators,
         final_estimator=final_estimator)

In [ ]:
from sklearn.model_selection import KFold

accuracy=[]
prediction2=0

skf=KFold(n_splits=5, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       reg2.fit(X1_train,Y1_train)
       prediction2+=reg2.predict(df1)/5

prediction2=pd.DataFrame(prediction2)  
prediction2

In [ ]:
prediction2.to_csv('Sub17.csv',header=['prediction'],index=False)

In [ ]:
lgb1={'lambda_l1': 0.003005391918043113,
 'lambda_l2': 8.006108609364085e-07,
 'num_leaves': 166,
 'feature_fraction': 0.8470174798951228,
 'bagging_fraction': 0.9267780669292209,
 'bagging_freq': 7,
 'min_child_samples': 17}

In [ ]:
rat_params1={'n_estimators': 1000, 'max_depth': 17, 'max_samples': 0.8, 'max_features': 3}

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rag = RandomForestRegressor(n_estimators= 1000, max_depth= 17, max_samples= 0.8, max_features=3)

from sklearn.model_selection import KFold

accuracy=[]
prediction_rat=0

skf=KFold(n_splits=5, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       rag.fit(X1_train,Y1_train)
       prediction_rat+=rag.predict(df1)/5

#prediction_rat=pd.DataFrame(prediction1)  
#prediction1

In [ ]:
lag=lgbm.LGBMRegressor(device_type='gpu',**lgb1)

from sklearn.model_selection import KFold

accuracy=[]
prediction_lgb=0

skf=KFold(n_splits=10, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       lag.fit(X1_train,Y1_train)
       prediction_lgb+=lag.predict(df1)/10

#prediction=pd.DataFrame(prediction1)  
#prediction1

In [ ]:
xgb1=XGBRegressor(tree_method='gpu_hist', **XGB_params)

from sklearn.model_selection import KFold

accuracy=[]
prediction_xgb=0

skf=KFold(n_splits=10, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       xgb1.fit(X1_train,Y1_train)
       prediction_xgb+=xgb1.predict(df1)/10

#prediction1=pd.DataFrame(prediction1)  
#prediction1

In [ ]:
cat=CatBoostRegressor(verbose=0, task_type='GPU', **catboost_params)

from sklearn.model_selection import KFold

accuracy=[]
prediction_cat=0

skf=KFold(n_splits=10, random_state=None)

for train_index, test_index in skf.split(X,y):
       X1_train,X1_test=X.iloc[train_index], X.iloc[test_index]
       Y1_train,Y1_test=y.iloc[train_index], y.iloc[test_index]

       cat.fit(X1_train,Y1_train)
       prediction_cat+=cat.predict(df1)/10

#prediction_cat=pd.DataFrame(prediction1)  
#prediction_cat

In [ ]:
final_avg=(0.2*prediction_rat+0.6*prediction_lgb+0.1*prediction_xgb+0.1*prediction_cat)
final_avg=pd.DataFrame(final_avg)
final_avg


In [ ]:
final_avg.to_csv('Sub54.csv',header=['prediction'],index=False)